In [1]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
"""
Functions definition
"""


def cal_Su_list(first_time_zero, time_end, dt=0.01):
    """
    Calculate the survival probability.

    Parameters:
    - first_time_zero (array-like): Array of first passage times.
    - time_end (float): The end time for calculating the survival probability.
    - dt (float, optional): Time step for the calculation. Default is 0.01.

    Returns:
    - ts (numpy.ndarray): Array of time points.
    - survival_probability (numpy.ndarray): Array of survival probabilities corresponding to the time points.
    """
    time_range = np.arange(0, time_end, dt)
    survival_probability = np.array([np.count_nonzero(first_time_zero >= t) / len(first_time_zero) for t in time_range])
    return time_range, survival_probability

def exp_fun(x, k, t0):
    """
    Exponential function used for fitting survival probability.

    Parameters:
    - x (numpy.ndarray): Time variable.
    - k (float): Rate constant.
    - t0 (float): Time offset.

    Returns:
    - numpy.ndarray: The value of the exponential function at time `x`.
    """
    t = x - t0
    t[t < 0] = 0
    return np.exp(-k * t)

def quadratic_fun(x, a, b, c):
    """
    Quadratic function for fitting purposes.

    Parameters:
    - x (numpy.ndarray): Independent variable (1/T).
    - a (float): Quadratic term coefficient.
    - b (float): Linear term coefficient.
    - c (float): Constant term.

    Returns:
    - numpy.ndarray: The value of the quadratic function at `x`.
    """
    return a * x**2 + b * x + c

def get_ln_kapp(FPT):
    """
    Calculate the natural logarithm of the apparent rate (ln_kapp) from the first passage time (FPT) array.

    Parameters:
    - FPT (array-like): First passage time array.

    Returns:
    - float: The natural logarithm of the apparent rate (ln_kapp).
    """
    time_end = np.max(FPT) + 5
    ts, Su_list = cal_Su_list(FPT, time_end)
    kopt, _ = curve_fit(exp_fun, ts, Su_list, bounds=(0, np.inf), maxfev=5000)
    k, t0 = kopt[0], kopt[1]
    kapp = 1 / (t0 + 1 / k)
    return np.log(kapp)


def write_SU(FPT, Temp):
    time_end = np.max(FPT) + 5
    ts, Su_list = cal_Su_list(FPT, time_end)
    with open(f'survival_{Temp}.dat', 'w') as f:
        for t, s in zip(ts, Su_list):
            f.write(f'{t:.2f}    {s:.3f}\n')

            

            
def bootstrap_FPT_dict(Temperature_FPT_dict):
    # Create a new dictionary for the random choices
    random_FPT_dict = {}

    # Apply np.random.choice with replacement to each array in the original dictionary
    for key in Temperature_FPT_dict:
        random_FPT_dict[key] = np.random.choice(Temperature_FPT_dict[key], size=len(Temperature_FPT_dict[key]), replace=True)

    # Print the new dictionary with random choices
#     print(random_FPT_dict)
    return random_FPT_dict

def extrapolate_ln_kapp_target_temperature(temperature_list, Temperature_FPT_dict, target_temp=298):
    """
    Extrapolates the natural logarithm of kapp (ln(kapp)) for a target temperature using quadratic fitting.
    
    Parameters:
    temperature_list (list of float): List of temperatures to be used for fitting.
    Temperature_FPT_dict (dict): Dictionary containing FPT values for each temperature. Keys should be formatted as 'FPT_<temperature>'.
    target_temp (float, optional): The target temperature for which ln(kapp) is to be extrapolated. Default is 298 K.
    
    Returns:
    tuple: 
        - numpy.ndarray: Inverse temperatures including the target temperature.
        - numpy.ndarray: ln(kapp) values including the extrapolated value for the target temperature.
        - float: Coefficient of determination (R^2) for the quadratic fit.
        - numpy.ndarray: Coefficients of the quadratic function (a, b, c).
    """
    # Initialize an array to store inverse temperatures and ln(kapp) values
    num_temperatures = len(temperature_list)
    inv_temperature_ln_kapp = np.zeros((num_temperatures + 1, 2))
    
    for i, temperature in enumerate(temperature_list):
        inv_temperature_ln_kapp[i, 0] = 1/temperature
        inv_temperature_ln_kapp[i, 1] = get_ln_kapp(Temperature_FPT_dict[f'FPT_{temperature}'])
        
    # Fit a quadratic function to the data points (excluding the target temperature)
    kopt_target, _ = curve_fit(quadratic_fun, inv_temperature_ln_kapp[:-1, 0], inv_temperature_ln_kapp[:-1, 1], maxfev=5000)
    
    # Calculate the coefficient of determination (R^2) for the fit
    predicted_ln_kapp = quadratic_fun(inv_temperature_ln_kapp[:-1, 0], *kopt_target)
    cc_target = np.corrcoef(inv_temperature_ln_kapp[:-1, 1], predicted_ln_kapp)[0, 1] ** 2
    
    # Extrapolate ln(kapp) for the target temperature
    ln_kapp_target = quadratic_fun(1 / target_temp, *kopt_target)

    # Add the target temperature and its extrapolated ln(kapp) value to the array
    inv_temperature_ln_kapp[-1,:] = 1/target_temp, ln_kapp_target
    return inv_temperature_ln_kapp[:, 0], inv_temperature_ln_kapp[:, 1], cc_target, kopt_target


def convert_ln_kapp_to_human(ln_kapp, human_type='day'):
    """
    Convert the logarithm of kapp (ln_kapp) to human time.

    Parameters:
    - ln_kapp (float): The natural logarithm of kapp.
    - human_type (str): The desired unit of human time. Can be 'second', 'hour', or 'day'. Default is 'day'.

    Returns:
    - float: The converted time in the specified human time unit.
    """
    scaled_factor = 143
    
    # Calculate the simulated life time in nanoseconds
    simulated_life_time = 1 / np.exp(ln_kapp)
    
    # Convert simulated life time from nanoseconds to human time
    conversion_factors = {
        'second': 1,
        'hour': 1 / 3600,
        'day': 1 / 86400
    }
    
    if human_type not in conversion_factors:
        raise ValueError("Invalid human_type. Expected one of: 'second', 'hour', 'day'")

    return scaled_factor * simulated_life_time / 1e9 * conversion_factors[human_type]


In [3]:
convert_ln_kapp_to_human(-26.32654157014862, 'day')

0.4490493028661993

In [4]:
convert_ln_kapp_to_human(-42.636606600510916, 'day')

5440831.405414913

In [5]:
5440831.405414913/365

14906.387412095652